In [1]:
from llama_index.core import (VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage
    , Document, Settings, StorageContext, PromptTemplate)
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.extractors import KeywordExtractor, SummaryExtractor
from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.dashscope import DashScope

from llama_index.extractors.entity import EntityExtractor
from llama_index.readers.file import UnstructuredReader,PyMuPDFReader

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

import os, re, asyncio
from tqdm.asyncio import tqdm_asyncio
from tqdm import tqdm
import json

In [2]:
embedding_model = "./Qwen3-Embedding-0.6B"
Settings.embed_model = HuggingFaceEmbedding(
    model_name=embedding_model,
    cache_folder=None,
    trust_remote_code=True,
    local_files_only=True
)

config = AutoConfig.from_pretrained(embedding_model, trust_remote_code=True, local_files_only=True)
dimension = config.hidden_size
print(f"模型嵌入维度: {dimension}")

2025-09-30 16:39:57,335 - INFO - Load pretrained SentenceTransformer: ./Qwen3-Embedding-0.6B
2025-09-30 16:39:58,539 - INFO - 1 prompt is loaded, with the key: query


模型嵌入维度: 1024


In [3]:
# main_model = "./Qwen3-4B-Thinking-2507"
# Settings.llm = HuggingFaceLLM(
#     model_name=main_model,
#     tokenizer_name=main_model,
#     generate_kwargs={"temperature": 0.1, "top_p": 0.7},
#     device_map="cuda",
#     max_new_tokens=512
# )

Settings.llm = DashScope(
    api_key="sk-7afc6caf37b64d069ef3be129e68753a",
    model="qwen3-max",
    generate_kwargs={"temperature": 0.1, "top_p": 0.7},
    max_new_tokens=512
)

In [4]:
milvus_dir = "./milvus_test"
milvus_db_path = os.path.join(milvus_dir, "milvus_lite.db")
abs_db_path = os.path.abspath(milvus_db_path)
print(f"绝对数据库路径: {abs_db_path}")

if not os.path.exists(milvus_dir):
    os.makedirs(milvus_dir)
    print("已创建 ./milvus 目录")

milvus_vector_store = MilvusVectorStore(
    uri=f"{abs_db_path}",
    collection_name="rag_collection",
    dim=1024,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=milvus_vector_store)

绝对数据库路径: /root/marathon/milvus_test/milvus_lite.db
已创建 ./milvus 目录


/root/miniconda3/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [5]:
def clean_text(text: str) -> str:
    text = re.sub(r'\n\s*\n+', '\n\n', text).strip()
    # text = re.sub(r'(\w+\s*){3,}\n', '', text)
    # text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fa5\s\.,!?]', '', text)  # 去除特殊字符，保留中英文
    return text


In [14]:
async def generate_summary_async(text, max_words=20):
    prompt = f"总结以下文本，不超过{max_words}字，直接回复结果：{text}"
    response = await Settings.llm.acomplete(prompt)
    return response.text.strip()

def generate_summary(text, max_words=20):
    prompt = f"总结以下文本，不超过{max_words}字，直接回复结果：{text}"
    response = Settings.llm.complete(prompt)
    return response.text.strip()

async def add_summaries_to_nodes_async(nodes_list):
    tasks = [generate_summary_async(node.text) for node in nodes_list]

    summaries = []
    for future in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="生成节点摘要进度"):
        summary = await future
        summaries.append(summary)

    for node, summary in zip(nodes_list, summaries):
        node.metadata["node_summary"] = summary
        
def add_summaries_to_nodes(nodes_list):
    for node in tqdm(nodes_list, desc="生成摘要"):
        summary = generate_summary(node.text)
        node.metadata["node_summary"] = summary


In [6]:
qwen_tokenizer = AutoTokenizer.from_pretrained("./Qwen3-Embedding-0.6B", trust_remote_code=True)
documents_dir = "./docs"

file_extractor = {
    ".pdf": PyMuPDFReader(), 
    ".docx": UnstructuredReader()
}
reader = SimpleDirectoryReader(input_dir=documents_dir, recursive=True, file_extractor=file_extractor)
documents = reader.load_data()

cleaned_documents = [Document(text=clean_text(doc.text), metadata=doc.metadata) for doc in documents]
documents = cleaned_documents

print(f"文件大小:{len(documents)}")

node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100, tokenizer=qwen_tokenizer.tokenize)
#nodes = node_parser.get_nodes_from_documents(documents)
#print(f"节点数量:{len(nodes)}")

#asyncio.run(add_summaries_to_nodes_async(nodes))
#add_summaries_to_nodes(nodes)

文件大小:326


In [10]:
def save_summaries_to_json(nodes_list, file_path="nodes_summaries_temp.json"):
    summaries_dict = {}
    for idx, node in enumerate(nodes_list):
        summaries_dict[str(idx)] = node.metadata.get("node_summary", "")  # 获取摘要，若无则为空
    
    # 保存到 JSON
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(summaries_dict, f, ensure_ascii=False, indent=4)
    
    print(f"节点摘要已保存到 {file_path}")

def load_summaries_to_nodes(nodes_list, file_path="nodes_summaries.json"):
    with open(file_path, 'r', encoding='utf-8') as f:
        summaries_dict = json.load(f)
    sorted_keys = sorted(summaries_dict.keys(), key=int)

    for key in sorted_keys:
        idx = int(key)
        if idx < len(nodes_list):
            nodes_list[idx].metadata["node_summary"] = summaries_dict[key]
        else:
            print(f"警告：索引 {idx} 超出节点列表长度，跳过。")
    
    return nodes_list


In [40]:
save_summaries_to_json(nodes)

节点摘要已保存到 nodes_summaries.json


In [32]:
# keyword_extractor = KeywordExtractor(
#     llm=Settings.llm,  # 使用 LLM 提取关键词作为主题
#     keywords=5,  # 提取前 5 个关键词
#     prompt_template_str="""
#     从以下文本中提取 2 个主要关键词。
#     输出格式：仅用逗号分隔的关键词列表，不要添加任何解释或额外文本。
#     示例输出：关键词1,关键词2

#     文本：{text}
#     """
# )
# max_words = 20
# summary_extractor = SummaryExtractor(
#     llm=Settings.llm,
#     summaries=["self"],
#     prompt_template_str="""
#     总结以下文本，不超过20字，直接回复结果：\n{text}
#         """
# )


In [7]:
# index = VectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     embed_model=Settings.embed_model,
#     node_parser=node_parser,
#     store_nodes_override=True
# )



transformations = [node_parser]
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=Settings.embed_model,
    # node_parser=node_parser,
    transformations=transformations,
    store_nodes_override=True
)


In [8]:
nodes = list(index.docstore.docs.values())
print(len(documents[0].text))
print(f"节点数量: {len(nodes)}")


108
节点数量: 353


In [11]:
loaded_nodes = load_summaries_to_nodes(nodes)

In [12]:
index = VectorStoreIndex(
    loaded_nodes, 
    storage_context=storage_context, 
    embed_model=Settings.embed_model, 
    store_nodes_override=True)

In [13]:
nodes_with_sums = list(index.docstore.docs.values())

In [15]:
for node in nodes_with_sums:
    print(node.metadata["node_summary"])

李宏毅深度学习教程1.2.4版，2025年发布。
李宏毅老师的《机器学习》课程幽默易懂，适合深度学习入门。
三位作者简介及成就概述。
数学与机器学习中常用符号及其含义说明。
书籍介绍机器学习基础与实践方法论。
深度学习基础：局部极小值、批量、动量及自适应学习率。
学习率调度、优化总结、分类及批量归一化等内容介绍。
卷积神经网络通过感受野检测局部模式，适用于图像各区域。
卷积与循环神经网络简介及应用
RNN、双向循环神经网络及LSTM原理与应用。
自注意力机制及其在Transformer中的应用。
第7章介绍Transformer及其在多种任务中的应用。
Transformer结构及其编码器、解码器和训练过程介绍。
文本讨论了引导注意力、束搜索等技术及生成对抗网络等内容。
GAN理论、WGAN算法及训练难点，扩散模型与自监督学习简介。
介绍生成式预训练模型和自编码器的概念及用途。
自编码器应用与对抗攻击简介
介绍了被动防御、主动防御及迁移学习和强化学习的概念与应用。
介绍未知函数、定义损失、优化及多种评价动作标准和元学习概念。
元学习的步骤、与机器学习关系、算法及应用。
书籍内容涵盖终身学习、网络压缩及可解释性AI。
讨论了决策树可解释性、ChatGPT及Minecraft纯视觉研究。
介绍了一种新的图计算方法及其在行为学习中的应用。
机器学习基础介绍及分类、回归和结构化学习概述。
机器学习中模型参数估计与损失函数定义。
通过计算每日预测误差并求平均得到损失L。
机器学习中通过调整参数w和b来最小化损失L。
通过计算损失函数的偏导数调整参数w以最小化损失。
梯度下降可能陷入局部最小值，而非全局最小值。
梯度下降法更新w和b以最小化损失L。
观看人次预测模型从考虑1天改进到考虑7天，误差降低。
线性模型预测观看人次，考虑天数增加可降低损失。
蓝线斜坡与红线斜坡对齐，通过叠加得到红线形状。
分段线性曲线可由多个蓝色函数组合逼近任何连续曲线。
! 欢![](https://i.imgur.com/1![](https://i.imgur.com/) Sigmoid函数可以![](https://i.imgur.com/)

看起来![](https://i.imgur.com/)

看起来您提供的文本中包含了一些不连贯的信息和排版错误，我尝试整理并总结了关键点![](htt